<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-nc-sa/4.0/80x15.png" /></a>&nbsp;| [Emmanuel Rachelson](https://personnel.isae-supaero.fr/emmanuel-rachelson?lang=en) | <a href="https://supaerodatascience.github.io/machine-learning/">https://supaerodatascience.github.io/machine-learning/</a>

<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">Text data pre-processing</div>

In this exercice, we shall load a database of email messages and pre-format them so that we can design automated classification methods or use off-the-shelf classifiers. The general purpose of this notebook is to give a practical notion (through this example) of how important data pre-processing can be in a Machine Learning workflow, and generalize it to other situations.

"What is there to pre-process?" you might ask. Well, actually, text data comes in a very noisy form that we, humans, have become accustomed to and filter out effortlessly to grasp the core meaning of the text. It has a lot of formatting (fonts, colors, typography...), punctuation, abbreviations, common words, grammatical rules, etc. that we might wish to discard before even starting the data analysis.

Here are some pre-processing steps that can be performed on text:
1. loading the data, removing attachements, merging title and body;
2. tokenizing - splitting the text into atomic "words";
3. removal of stop-words - very common words;
4. removal of non-words - punctuation, numbers, gibberish;
3. lemmatization - merge together "find", "finds", "finder".

The final goal is to be able to represent a document as a mathematical object, e.g. a vector, that our machine learning black boxes can process.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-Load-the-data" data-toc-modified-id="1.-Load-the-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>1. Load the data</a></span></li><li><span><a href="#2.-Filtering-out-the-noise" data-toc-modified-id="2.-Filtering-out-the-noise-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>2. Filtering out the noise</a></span></li><li><span><a href="#3.-Even-better-filtering" data-toc-modified-id="3.-Even-better-filtering-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>3. Even better filtering</a></span></li><li><span><a href="#4.-Term-frequency-times-inverse-document-frequency" data-toc-modified-id="4.-Term-frequency-times-inverse-document-frequency-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>4. Term frequency times inverse document frequency</a></span></li><li><span><a href="#5.-Utility-function" data-toc-modified-id="5.-Utility-function-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>5. Utility function</a></span></li></ul></div>

# 1. Load the data

Let's first load the emails.

In [1]:
import os
data_switch=1
if(data_switch==0):
    train_dir = '../data/ling-spam/train-mails/'
    email_path = [os.path.join(train_dir,f) for f in os.listdir(train_dir)]
else:
    train_dir = '../data/lingspam_public/bare/'
    email_path = []
    email_label = []
    for d in os.listdir(train_dir):
        folder = os.path.join(train_dir,d)
        email_path += [os.path.join(folder,f) for f in os.listdir(folder)]
        email_label += [f[0:3]=='spm' for f in os.listdir(folder)]
print("number of emails",len(email_path))
email_nb = 8 # try 8 for a spam example
print("email file:", email_path[email_nb])
print("email is a spam:", email_label[email_nb])
print(open(email_path[email_nb]).read())

number of emails 2893
email file: ../data/lingspam_public/bare/part1\3-378msg5.txt
email is a spam: False
Subject: t

hi , help ! i have to design an experiment to do with mandarin tones as part of a phonology requirement on my graduate course . there seems to be very little literature on this in the library . if anyone can think of any on-going debates on the phonology / phonetics of mandarin tones for which an experiment would be useful , please could you give me information and references . i would welcome any suggestions at all . thanks a lot , sophia wang . ( sophia @ ling . ed . ac . uk )



# 2. Filtering out the noise

One nice thing about scikit-learn is that is has lots of preprocessing utilities. Like [`CountVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) for instance, that converts a collection of text documents to a matrix of token counts.

- To remove stop-words, we set: `stop_words='english'`
- To convert all words to lowercase: `lowercase=True`
- The default tokenizer in scikit-learn removes punctuation and only keeps words of more than 2 letters.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
countvect = CountVectorizer(input='filename', stop_words='english', lowercase=True)
word_count = countvect.fit_transform(email_path)

In [3]:
print("Number of documents:", len(email_path))
words = countvect.get_feature_names()
print("Number of words:", len(words))
print("Document - words matrix:", word_count.shape)
print("First words:", words[0:100])

Number of documents: 2893
Number of words: 60618
Document - words matrix: (2893, 60618)
First words: ['00', '000', '0000', '00001', '00003000140', '00003003958', '00007', '0001', '00010', '00014', '0003', '00036', '000bp', '000s', '000yen', '001', '0010', '0010010034', '0011', '00133', '0014', '00170', '0019', '00198', '002', '002656', '0027', '003', '0030', '0031', '00333', '0037', '0039', '003n7', '004', '0041', '0044', '0049', '005', '0057', '006', '0067', '007', '00710', '0073', '0074', '00799', '008', '009', '00919680', '0094', '00a', '00am', '00arrival', '00b', '00coffee', '00congress', '00d', '00dinner', '00f', '00h', '00hfstahlke', '00i', '00j', '00l', '00m', '00p', '00pm', '00r', '00t', '00tea', '00the', '00uzheb', '01', '0100', '01003', '01006', '0104', '0106', '01075', '0108', '011', '0111', '0117', '0118', '01202', '01222', '01223', '01225', '01232', '01235', '01273', '013', '0131', '01334', '0135', '01364', '0139', '013953', '013a']


C:\Users\User\anaconda3\envs\algoMachineLearning\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# 3. Even better filtering

That's already quite ok, but this pre-processing does not perform lemmatization, the list of stop-words could be better and we could wish to remove non-english words (misspelled, with numbers, etc.).

A slightly better preprocessing uses the [Natural Language Toolkit](https://www.nltk.org/https://www.nltk.org/). The one below:
- tokenizes;
- removes punctuation;
- removes stop-words;
- removes non-English and misspelled words (optional);
- removes 1-character words;
- removes non-alphabetical words (numbers and codes essentially).

In [4]:
from nltk import wordpunct_tokenize          
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import words
from string import punctuation
class LemmaTokenizer(object):
    def __init__(self, remove_non_words=True):
        self.wnl = WordNetLemmatizer()
        self.stopwords = set(stopwords.words('english'))
        self.words = set(words.words())
        self.remove_non_words = remove_non_words
    def __call__(self, doc):
        # tokenize words and punctuation
        word_list = wordpunct_tokenize(doc)
        # remove stopwords
        word_list = [word for word in word_list if word not in self.stopwords]
        # remove non words
        if(self.remove_non_words):
            word_list = [word for word in word_list if word in self.words]
        # remove 1-character words
        word_list = [word for word in word_list if len(word)>1]
        # remove non alpha
        word_list = [word for word in word_list if word.isalpha()]
        return [self.wnl.lemmatize(t) for t in word_list]

countvect = CountVectorizer(input='filename',tokenizer=LemmaTokenizer(remove_non_words=True))
word_count = countvect.fit_transform(email_path)
feat2word = {v: k for k, v in countvect.vocabulary_.items()}

In [5]:
print("Number of documents:", len(email_path))
words = countvect.get_feature_names()
print("Number of words:", len(words))
print("Document - words matrix:", word_count.shape)
print("First words:", words[0:100])

Number of documents: 2893
Number of words: 14279
Document - words matrix: (2893, 14279)
First words: ['aa', 'aal', 'aba', 'aback', 'abacus', 'abandon', 'abandoned', 'abandonment', 'abbas', 'abbreviation', 'abdomen', 'abduction', 'abed', 'aberrant', 'aberration', 'abide', 'abiding', 'abigail', 'ability', 'ablative', 'ablaut', 'able', 'abler', 'aboard', 'abolition', 'abord', 'aboriginal', 'aborigine', 'abound', 'abox', 'abreast', 'abridged', 'abroad', 'abrogate', 'abrook', 'abruptly', 'abscissa', 'absence', 'absent', 'absolute', 'absolutely', 'absoluteness', 'absolutist', 'absolutive', 'absolutization', 'absorbed', 'absorption', 'abstract', 'abstraction', 'abstractly', 'abstractness', 'absurd', 'absurdity', 'abu', 'abundance', 'abundant', 'abuse', 'abusive', 'abyss', 'academe', 'academic', 'academically', 'academician', 'academy', 'accelerate', 'accelerated', 'accelerative', 'accent', 'accentuate', 'accentuation', 'accept', 'acceptability', 'acceptable', 'acceptance', 'acceptation', 'acc

# 4. Term frequency times inverse document frequency

After this first preprocessing, each document is summarized by a vector of size "number of words in the extracted dictionnary". For example, the first email in the list has become:

In [6]:
mail_number = 0
text = open(email_path[mail_number]).read()
print("Original email:")
print(text)
#print(LemmaTokenizer()(text))
#print(len(set(LemmaTokenizer()(text))))
#print(len([feat2word[i] for i in word_count2[mail_number, :].nonzero()[1]]))
#print(len([word_count2[mail_number, i] for i in word_count2[mail_number, :].nonzero()[1]]))
#print(set([feat2word[i] for i in word_count2[mail_number, :].nonzero()[1]])-set(LemmaTokenizer()(text)))
emailBagOfWords = {feat2word[i]: word_count[mail_number, i] for i in word_count[mail_number, :].nonzero()[1]}
print("Bag of words representation (", len(emailBagOfWords), " words in dict):", sep='')
print(emailBagOfWords)
print("\nVector reprensentation (", word_count[mail_number, :].nonzero()[1].shape[0], " non-zero elements):", sep='')
print(word_count[mail_number, :])

Original email:
Subject: re : 2 . 882 s - > np np

> date : sun , 15 dec 91 02 : 25 : 02 est > from : michael < mmorse @ vm1 . yorku . ca > > subject : re : 2 . 864 queries > > wlodek zadrozny asks if there is " anything interesting " to be said > about the construction " s > np np " . . . second , > and very much related : might we consider the construction to be a form > of what has been discussed on this list of late as reduplication ? the > logical sense of " john mcnamara the name " is tautologous and thus , at > that level , indistinguishable from " well , well now , what have we here ? " . to say that ' john mcnamara the name ' is tautologous is to give support to those who say that a logic-based semantics is irrelevant to natural language . in what sense is it tautologous ? it supplies the value of an attribute followed by the attribute of which it is the value . if in fact the value of the name-attribute for the relevant entity were ' chaim shmendrik ' , ' john mcnamara the na

Counting words is a good start but there is an issue: longer documents will have higher average count values than shorter documents, even though they might talk about the same topics.

To avoid these potential discrepancies it suffices to divide the number of occurrences of each word in a document by the total number of words in the document: these new features are called `tf` for Term Frequencies.

Another refinement on top of `tf` is to downscale weights for words that occur in many documents in the corpus and are therefore less informative than those that occur only in a smaller portion of the corpus.

This downscaling is called `tf–idf` for “Term Frequency times Inverse Document Frequency” and again, scikit-learn does the job for us with the [TfidfTransformer](scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html) function.

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer().fit_transform(word_count)
tfidf.shape

(2893, 14279)

Now every email in the corpus has a vector representation that filters out unrelevant tokens and retains the significant information.

In [8]:
print("email 0:")
print(tfidf[0,:])

email 0:
  (0, 14188)	0.0476366389153867
  (0, 14001)	0.10863457040408879
  (0, 13666)	0.29378496199758364
  (0, 12800)	0.08202664607875251
  (0, 12566)	0.1724316642310435
  (0, 12565)	0.5172949926931305
  (0, 12307)	0.07916060019893302
  (0, 12273)	0.11037003020069999
  (0, 12153)	0.0438100675761268
  (0, 11226)	0.16647435384469672
  (0, 11207)	0.06780624737245437
  (0, 11143)	0.06148918450544994
  (0, 11030)	0.15092314041652832
  (0, 10938)	0.08904659377287111
  (0, 10436)	0.07875114839373085
  (0, 10418)	0.06660895299901055
  (0, 10313)	0.2519744396619634
  (0, 8215)	0.06881355283176444
  (0, 8176)	0.21942273948546862
  (0, 8094)	0.0643418459994976
  (0, 7837)	0.07090776914535644
  (0, 7349)	0.09996746952846733
  (0, 7348)	0.09199479564318516
  (0, 7283)	0.054992378193498954
  (0, 7162)	0.07546157020826416
  (0, 7043)	0.08872445511740758
  (0, 7019)	0.03736264352558252
  (0, 6718)	0.13576315389323418
  (0, 6566)	0.08451853830618253
  (0, 6321)	0.1813133911240336
  (0, 5345)	0.070907

# 5. Utility function

Let's put all this loading process into a separate file so that we can reuse it in other experiments.

In [9]:
import load_spam
spam_data = load_spam.spam_data_loader()
spam_data.load_data()

In [10]:
spam_data.print_email(8)

email file: ../data/lingspam_public/bare/part1\3-378msg5.txt
email is a spam: False
Subject: t

hi , help ! i have to design an experiment to do with mandarin tones as part of a phonology requirement on my graduate course . there seems to be very little literature on this in the library . if anyone can think of any on-going debates on the phonology / phonetics of mandarin tones for which an experiment would be useful , please could you give me information and references . i would welcome any suggestions at all . thanks a lot , sophia wang . ( sophia @ ling . ed . ac . uk )

Bag of words representation (30 words in dictionary):
{'anyone': 1, 'could': 1, 'course': 1, 'design': 1, 'experiment': 2, 'give': 1, 'going': 1, 'graduate': 1, 'help': 1, 'hi': 1, 'information': 1, 'library': 1, 'ling': 1, 'literature': 1, 'little': 1, 'lot': 1, 'mandarin': 2, 'part': 1, 'phonetics': 1, 'phonology': 2, 'please': 1, 'requirement': 1, 'sophia': 2, 'subject': 1, 'thanks': 1, 'think': 1, 'useful': 1, '